<a href="https://colab.research.google.com/github/JescYip/StyleTransfer/blob/main/train3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')
!bash init_style_transfer_v2.sh
!pip uninstall -y tensorboard tensorflow jax jaxlib
!pip install tensorboardX
!pip install lpips
%cd /content/StyleTransfer

Mounted at /content/drive
[⚠️ 提示] 请确保你已在 Colab Notebook 中执行以下代码挂载 Google Drive：
from google.colab import drive; drive.mount('/content/drive')

📥 正在克隆 StyleTransfer 仓库...
Cloning into 'StyleTransfer'...
remote: Enumerating objects: 194, done.
remote: Counting objects: 100% (194/194), done.
remote: Compressing objects: 100% (165/165), done.
remote: Total 194 (delta 98), reused 57 (delta 26), pack-reused 0 (from 0)
Receiving objects: 100% (194/194), 9.95 MiB | 7.79 MiB/s, done.
Resolving deltas: 100% (98/98), done.
📦 正在安装 PyTorch 和依赖库...
Looking in indexes: https://download.pytorch.org/whl/cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 GB 427.8 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 100.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 95.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.3/63.3 MB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.3/132.3 kB 12.6 MB/s eta 0:00:

In [ ]:
import sys
import os
import imageio
import numpy as np
import re

sys.path.append('/content/StyleTransfer')
from utils import rgb_lpips

device = 'cuda'
lpips_total = 0
rmse_total = 0
n = 0

# ✅ 预测图像命名为 001.png, 002.png...
pred_dir = '/content/drive/MyDrive/log_style/lego_style_starry/lego_style2/imgs_test_all/fdd94f1da0ee2034934c95ea6e3a184ec'
# ✅ ground truth 图像命名为 r_1.png, r_2.png...
gt_dir = '/content/drive/MyDrive/data/nerf_synthetic/lego/test'

def to_rgb(img):
    if img.shape[-1] == 4:
        return img[..., :3]
    return img

def compute_rmse(img1, img2):
    return np.sqrt(np.mean((img1 - img2) ** 2))

for fname in sorted(os.listdir(pred_dir)):
    if not fname.endswith('.png') or 'depth' in fname:
        continue

    try:
        idx = int(os.path.splitext(fname)[0])
        gt_fname = f"r_{idx}.png"

        pred_img = imageio.imread(os.path.join(pred_dir, fname)).astype(np.float32) / 255.0
        gt_img = imageio.imread(os.path.join(gt_dir, gt_fname)).astype(np.float32) / 255.0

        pred_img = to_rgb(pred_img)
        gt_img = to_rgb(gt_img)

        lpips_val = rgb_lpips(gt_img, pred_img, 'vgg', device)
        rmse_val = compute_rmse(gt_img, pred_img)

        lpips_total += lpips_val
        rmse_total += rmse_val
        n += 1

    except Exception as e:
        print(f"跳过 {fname}，出错原因：{e}")
        continue

if n > 0:
    print(f"\n✅ Avg LPIPS over {n} images: {lpips_total / n:.4f}")
    print(f"✅ Avg RMSE  over {n} images: {rmse_total / n:.4f}")
else:
    print("\n⚠️ 没有成功处理任何图像，请检查文件命名或路径是否正确。")


<ipython-input-9-14e5ef670688>:36: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  pred_img = imageio.imread(os.path.join(pred_dir, fname)).astype(np.float32) / 255.0
<ipython-input-9-14e5ef670688>:37: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  gt_img = imageio.imread(os.path.join(gt_dir, gt_fname)).astype(np.float32) / 255.0



✅ Avg LPIPS over 200 images: 0.4437
✅ Avg RMSE  over 200 images: 0.8506


In [ ]:
import sys
import os
import imageio.v2 as imageio
import numpy as np

sys.path.append('/content/StyleTransfer')
from utils import rgb_lpips

device = 'cuda'
lpips_total = 0
n = 0

# ✅ 预测图像命名为 001.png, 002.png...
pred_dir = '/content/drive/MyDrive/log/lego_base/imgs_test_all'
# ✅ ground truth 图像命名为 r_1.png, r_2.png...
gt_dir = '/content/drive/MyDrive/data/nerf_synthetic/lego/test'

def to_rgb(img):
    if img.shape[-1] == 4:
        return img[..., :3]
    return img

def check_image_pair(img1, img2, verbose=False):
    if verbose:
        print(f"尺寸: {img1.shape} vs {img2.shape}")
        if img1.shape != img2.shape:
            print("❌ 尺寸不一致")
        if img1.shape[-1] != 3:
            print("⚠️ 图像通道不是 RGB")

        print(f"归一化范围: {img1.min():.2f} ~ {img1.max():.2f}")
        if img1.max() > 1.0:
            print("⚠️ 图像未归一化到 [0, 1]")

lpips_vgg_total = 0
n = 0

for fname in sorted(os.listdir(pred_dir)):
    if not fname.endswith('.png') or 'depth' in fname:
        continue

    try:
        idx = int(os.path.splitext(fname)[0])
        gt_fname = f"r_{idx}.png"

        pred_img = imageio.imread(os.path.join(pred_dir, fname)).astype(np.float32) / 255.0
        gt_img = imageio.imread(os.path.join(gt_dir, gt_fname)).astype(np.float32) / 255.0

        pred_img = to_rgb(pred_img)
        gt_img = to_rgb(gt_img)

        lpips_vgg = rgb_lpips(gt_img, pred_img, 'vgg', device)
        lpips_vgg_total += lpips_vgg
        n += 1

    except Exception as e:
        print(f"跳过 {fname}，出错原因：{e}")
        continue

if n > 0:
    print(f"\n✅ Avg LPIPS (VGG)  over {n} images: {lpips_vgg_total / n:.4f}")
else:
    print("\n⚠️ 没有成功处理任何图像，请检查文件命名或路径是否正确。")


init_lpips: lpips_alex
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth
100%|██████████| 233M/233M [00:01<00:00, 204MB/s]


Loading model from: /usr/local/lib/python3.11/dist-packages/lpips/weights/v0.1/alex.pth

✅ Avg LPIPS (VGG)  over 200 images: 0.2730
✅ Avg LPIPS (Alex) over 200 images: 0.4977


In [ ]:
import lpips
import cv2
import torch
import torchvision.transforms as transforms
import torch.nn.functional as F

# 模型
loss_fn = lpips.LPIPS(net='vgg')  # or 'alex'

# 转换器
transform = transforms.ToTensor()
def load_img(path):
    img = cv2.imread(path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    transform = transforms.ToTensor()
    img = transform(img).unsqueeze(0)  # [1, 3, H, W]
    img = img * 2 - 1  # 归一化到 [-1, 1]
    return img

img1 = load_img('/content/drive/MyDrive/log_feature1/trex_feature_lowmem12/trex_feature_lowmem22/imgs_path_all/000.png')
img2 = load_img('/content/drive/MyDrive/data/nerf_llff_data/trex/images/DJI_20200223_163548_810.jpg')


H = min(img1.shape[2], img2.shape[2])
W = min(img1.shape[3], img2.shape[3])
img1 = F.interpolate(img1, size=(H, W), mode='bilinear', align_corners=False)
img2 = F.interpolate(img2, size=(H, W), mode='bilinear', align_corners=False)

# 计算 LPIPS
d = loss_fn(img1, img2)
print(f"LPIPS: {d.item():.4f}")



Setting up [LPIPS] perceptual loss: trunk [vgg], v[0.1], spatial [off]
Loading model from: /usr/local/lib/python3.11/dist-packages/lpips/weights/v0.1/vgg.pth
LPIPS: 0.2494


In [ ]:
import lpips
import cv2
import torch
import torchvision.transforms as transforms
import torch.nn.functional as F

# 模型
loss_fn = lpips.LPIPS(net='vgg')  # or 'alex'

# 转换器
transform = transforms.ToTensor()
def load_img(path):
    img = cv2.imread(path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    transform = transforms.ToTensor()
    img = transform(img).unsqueeze(0)  # [1, 3, H, W]
    img = img * 2 - 1  # 归一化到 [-1, 1]
    return img

img1 = load_img('/content/drive/MyDrive/log_feature1/trex_feature_lowmem12/fortress_feature/imgs_test_all/002.png')
img2 = load_img('/content/drive/MyDrive/data/nerf_llff_data/trex/images/DJI_20200223_163648_601.jpg')


H = min(img1.shape[2], img2.shape[2])
W = min(img1.shape[3], img2.shape[3])
img1 = F.interpolate(img1, size=(H, W), mode='bilinear', align_corners=False)
img2 = F.interpolate(img2, size=(H, W), mode='bilinear', align_corners=False)

# 计算 LPIPS
d = loss_fn(img1, img2)
print(f"LPIPS: {d.item():.4f}")



Setting up [LPIPS] perceptual loss: trunk [vgg], v[0.1], spatial [off]
Loading model from: /usr/local/lib/python3.11/dist-packages/lpips/weights/v0.1/vgg.pth
LPIPS: 0.5473


In [ ]:
import sys
import os
import imageio.v2 as imageio
import numpy as np

sys.path.append('/content/StyleTransfer')
from utils import rgb_lpips

device = 'cuda'
lpips_total = 0
n = 0

# ✅ 预测图像命名为 001.png, 002.png...
pred_dir = '/content/drive/MyDrive/log/lego_base/imgs_test_all'
# ✅ ground truth 图像命名为 r_1.png, r_2.png...
gt_dir = '/content/drive/MyDrive/data/nerf_synthetic/lego/test'

def to_rgb(img):
    if img.shape[-1] == 4:
        return img[..., :3]
    return img

# 记录每张图的 LPIPS 值，找最小
best_lpips = float('inf')
best_pair = None

for fname in sorted(os.listdir(pred_dir)):
    if not fname.endswith('.png') or 'depth' in fname:
        continue

    try:
        idx = int(os.path.splitext(fname)[0])
        gt_fname = f"r_{idx}.png"

        pred_img = imageio.imread(os.path.join(pred_dir, fname)).astype(np.float32) / 255.0
        gt_img = imageio.imread(os.path.join(gt_dir, gt_fname)).astype(np.float32) / 255.0

        pred_img = to_rgb(pred_img)
        gt_img = to_rgb(gt_img)

        lpips_val = rgb_lpips(gt_img, pred_img, 'vgg', device)

        if lpips_val < best_lpips:
            best_lpips = lpips_val
            best_pair = (fname, gt_fname)

        lpips_total += lpips_val
        n += 1

    except Exception as e:
        print(f"跳过 {fname}，出错原因：{e}")
        continue

if n > 0:
    print(f"\n✅ 平均 LPIPS (VGG) over {n} images: {lpips_total / n:.4f}")
    print(f"🌟 最小 LPIPS 为 {best_lpips:.4f}，来自：{best_pair[0]} vs {best_pair[1]}")
else:
    print("\n⚠️ 没有成功处理任何图像，请检查文件命名或路径是否正确。")


init_lpips: lpips_vgg
Setting up [LPIPS] perceptual loss: trunk [vgg], v[0.1], spatial [off]
Loading model from: /usr/local/lib/python3.11/dist-packages/lpips/weights/v0.1/vgg.pth

✅ 平均 LPIPS (VGG) over 200 images: 0.2730
🌟 最小 LPIPS 为 0.2519，来自：164.png vs r_164.png


In [ ]:
import sys
import os
import imageio.v2 as imageio
import numpy as np
import cv2
sys.path.append('/content/StyleTransfer')
from utils import rgb_lpips

device = 'cuda'
lpips_total = 0
n = 0

# ✅ 预测图像目录（001.png, 002.png...）
pred_dir = '/content/drive/MyDrive/log/lego_base/imgs_test_all'
# ✅ Ground Truth 图像目录（无规律文件名，比如 DJI_...jpg）
gt_dir = '/content/drive/MyDrive/data/nerf_llff_data/trex/images'

def to_rgb(img):
    if img.shape[-1] == 4:
        return img[..., :3]
    return img

def resize_to_match(gt_img, pred_img):
    h = min(gt_img.shape[0], pred_img.shape[0])
    w = min(gt_img.shape[1], pred_img.shape[1])
    gt_img_resized = cv2.resize(gt_img, (w, h), interpolation=cv2.INTER_AREA)
    pred_img_resized = cv2.resize(pred_img, (w, h), interpolation=cv2.INTER_AREA)
    return gt_img_resized, pred_img_resized

# 获取并排序预测图和GT图文件名（确保文件数相同）
pred_files = sorted([f for f in os.listdir(pred_dir) if f.endswith('.png')])
gt_files = sorted([f for f in os.listdir(gt_dir) if f.endswith('.jpg')])

results = []

# 用于记录最小LPIPS


for pred_fname in tqdm(pred_files):
    try:
        pred_path = os.path.join(pred_dir, pred_fname)
        pred_img = imageio.imread(pred_path).astype(np.float32) / 255.0
        pred_img = to_rgb(pred_img)

        best_lpips = float('inf')
        best_gt_fname = None

        for gt_fname in gt_files:
            gt_path = os.path.join(gt_dir, gt_fname)
            gt_img = imageio.imread(gt_path).astype(np.float32) / 255.0
            gt_img = to_rgb(gt_img)
            gt_img, pred_img = resize_to_match(gt_img, pred_img)

            lpips_val = rgb_lpips(gt_img, pred_img, 'vgg', device)

            if lpips_val < best_lpips:
                best_lpips = lpips_val
                best_gt_fname = gt_fname

        print(f"{pred_fname} 最接近的是 {best_gt_fname}，LPIPS: {best_lpips:.4f}")
        results.append((pred_fname, best_gt_fname, best_lpips))
        lpips_total += best_lpips
        n += 1

    except Exception as e:
        print(f"跳过 {pred_fname}，出错原因：{e}")
        continue

if n > 0:
    print(f"\n✅ 平均最小 LPIPS（{n} 张预测图）: {lpips_total / n:.4f}")
else:
    print("\n⚠️ 没有成功处理任何图像，请检查路径或文件是否有效。")


  0%|          | 1/200 [00:50<2:47:14, 50.42s/it]

000.png 最接近的是 DJI_20200223_163548_810.jpg，LPIPS: 0.6968


  1%|          | 2/200 [01:12<1:50:52, 33.60s/it]

001.png 最接近的是 DJI_20200223_163548_810.jpg，LPIPS: 0.6963


  2%|▏         | 3/200 [01:35<1:34:10, 28.68s/it]

...

015.png 最接近的是 DJI_20200223_163548_810.jpg，LPIPS: 0.6853


  8%|▊         | 17/200 [06:43<1:07:10, 22.02s/it]

016.png 最接近的是 DJI_20200223_163548_810.jpg，LPIPS: 0.6852


  9%|▉         | 18/200 [07:05<1:06:25, 21.90s/it]

017.png 最接近的是 DJI_20200223_163548_810.jpg，LPIPS: 0.6851


In [2]:
!python train.py \
  --config configs/llff.txt \
  --datadir /content/drive/MyDrive/data/nerf_llff_data/orchids \
  --expname orchids_base


Namespace(config='configs/llff.txt', expname='orchids_base', basedir='./log', add_timestamp=0, datadir='/content/drive/MyDrive/data/nerf_llff_data/orchids', wikiartdir='./data/WikiArt', progress_refresh_rate=10, with_depth=False, downsample_train=4.0, downsample_test=1.0, model_name='TensorVMSplit', batch_size=4096, n_iters=25000, dataset_name='llff', patch_size=256, chunk_size=4096, lr_init=0.02, lr_basis=0.0001, lr_finetune=1e-05, lr_decay_iters=-1, lr_decay_target_ratio=0.1, lr_upsample_reset=1, L1_weight_inital=0.0, L1_weight_rest=0, Ortho_weight=0.0, TV_weight_density=1.0, TV_weight_app=1.0, TV_weight_feature=0.0, style_weight=0, content_weight=0, image_tv_weight=0, featuremap_tv_weight=0, n_lamb_sigma=[16, 4, 4], n_lamb_sh=[48, 12, 12], data_dim_color=27, rm_weight_mask_thre=0.0001, alpha_mask_thre=0.0001, distance_scale=25, density_shift=-10, shadingMode='MLP_Fea', pos_pe=6, view_pe=0, fea_pe=0, featureC=128, ckpt=None, render_only=0, render_test=1, render_train=0, render_path=1

In [4]:
!python /content/StyleTransfer/train_feature.py \
  --config /content/StyleTransfer/configs/llff_feature.txt \
  --datadir /content/drive/MyDrive/data/nerf_llff_data/orchids \
  --expname orchids_feature \
  --ckpt /content/drive/MyDrive/orchids_base/orchids_base.th \
  --basedir /content/drive/MyDrive/orchids_feature


Namespace(config='/content/StyleTransfer/configs/llff_feature.txt', expname='orchids_feature', basedir='/content/drive/MyDrive/orchids_feature', add_timestamp=0, datadir='/content/drive/MyDrive/data/nerf_llff_data/orchids', wikiartdir='./data/WikiArt', progress_refresh_rate=10, with_depth=False, downsample_train=4.0, downsample_test=1.0, model_name='TensorVMSplit', batch_size=4096, n_iters=10000, dataset_name='llff', patch_size=256, chunk_size=4096, lr_init=0.02, lr_basis=0.0001, lr_finetune=1e-05, lr_decay_iters=-1, lr_decay_target_ratio=0.1, lr_upsample_reset=1, L1_weight_inital=0.0, L1_weight_rest=0, Ortho_weight=0.0, TV_weight_density=0.0, TV_weight_app=0.0, TV_weight_feature=80.0, style_weight=0, content_weight=0, image_tv_weight=0, featuremap_tv_weight=0, n_lamb_sigma=[16, 4, 4], n_lamb_sh=[48, 12, 12], data_dim_color=27, rm_weight_mask_thre=0.0001, alpha_mask_thre=0.0001, distance_scale=25, density_shift=-10, shadingMode='MLP_Fea', pos_pe=6, view_pe=6, fea_pe=6, featureC=128, ck

In [3]:
!python /content/StyleTransfer/train.py \
  --config /content/StyleTransfer/configs/llff.txt \
  --datadir /content/drive/MyDrive/data/nerf_llff_data/orchids \
  --ckpt /content/StyleTransfer/log/orchids_base/orchids_base.th \
  --render_only 1 \
  --render_test 1

Namespace(config='/content/StyleTransfer/configs/llff.txt', expname='trex', basedir='./log', add_timestamp=0, datadir='/content/drive/MyDrive/data/nerf_llff_data/orchids', wikiartdir='./data/WikiArt', progress_refresh_rate=10, with_depth=False, downsample_train=4.0, downsample_test=1.0, model_name='TensorVMSplit', batch_size=4096, n_iters=25000, dataset_name='llff', patch_size=256, chunk_size=4096, lr_init=0.02, lr_basis=0.0001, lr_finetune=1e-05, lr_decay_iters=-1, lr_decay_target_ratio=0.1, lr_upsample_reset=1, L1_weight_inital=0.0, L1_weight_rest=0, Ortho_weight=0.0, TV_weight_density=1.0, TV_weight_app=1.0, TV_weight_feature=0.0, style_weight=0, content_weight=0, image_tv_weight=0, featuremap_tv_weight=0, n_lamb_sigma=[16, 4, 4], n_lamb_sh=[48, 12, 12], data_dim_color=27, rm_weight_mask_thre=0.0001, alpha_mask_thre=0.0001, distance_scale=25, density_shift=-10, shadingMode='MLP_Fea', pos_pe=6, view_pe=0, fea_pe=0, featureC=128, ckpt='/content/StyleTransfer/log/orchids_base/orchids_b

In [4]:
!zip -r log.zip /content/StyleTransfer/log
from google.colab import files
files.download("log.zip")

  adding: content/StyleTransfer/log/ (stored 0%)
  adding: content/StyleTransfer/log/orchids_base/ (stored 0%)
  adding: content/StyleTransfer/log/orchids_base/trex/ (stored 0%)
  adding: content/StyleTransfer/log/orchids_base/trex/imgs_path_all/ (stored 0%)
  adding: content/StyleTransfer/log/orchids_base/trex/imgs_path_all/057.png (deflated 0%)
  adding: content/StyleTransfer/log/orchids_base/trex/imgs_path_all/072.png (deflated 0%)
  adding: content/StyleTransfer/log/orchids_base/trex/imgs_path_all/063.png (deflated 0%)
  adding: content/StyleTransfer/log/orchids_base/trex/imgs_path_all/007.png (deflated 0%)
  adding: content/StyleTransfer/log/orchids_base/trex/imgs_path_all/042.png (deflated 0%)
  adding: content/StyleTransfer/log/orchids_base/trex/imgs_path_all/112.png (deflated 0%)
  adding: content/StyleTransfer/log/orchids_base/trex/imgs_path_all/018.png (deflated 0%)
  adding: content/StyleTransfer/log/orchids_base/trex/imgs_path_all/048.png (deflated 0%)
  adding: content/Sty

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!python train.py \
  --config configs/llff.txt \
  --datadir /content/drive/MyDrive/data/nerf_llff_data/fortress \
  --expname fortress


Namespace(config='configs/llff.txt', expname='fortress', basedir='./log', add_timestamp=0, datadir='/content/drive/MyDrive/data/nerf_llff_data/fortress', wikiartdir='./data/WikiArt', progress_refresh_rate=10, with_depth=False, downsample_train=4.0, downsample_test=1.0, model_name='TensorVMSplit', batch_size=4096, n_iters=25000, dataset_name='llff', patch_size=256, chunk_size=4096, lr_init=0.02, lr_basis=0.0001, lr_finetune=1e-05, lr_decay_iters=-1, lr_decay_target_ratio=0.1, lr_upsample_reset=1, L1_weight_inital=0.0, L1_weight_rest=0, Ortho_weight=0.0, TV_weight_density=1.0, TV_weight_app=1.0, TV_weight_feature=0.0, style_weight=0, content_weight=0, image_tv_weight=0, featuremap_tv_weight=0, n_lamb_sigma=[16, 4, 4], n_lamb_sh=[48, 12, 12], data_dim_color=27, rm_weight_mask_thre=0.0001, alpha_mask_thre=0.0001, distance_scale=25, density_shift=-10, shadingMode='MLP_Fea', pos_pe=6, view_pe=0, fea_pe=0, featureC=128, ckpt=None, render_only=0, render_test=1, render_train=0, render_path=1, e

In [ ]:
!python /content/StyleTransfer/train.py \
  --config /content/StyleTransfer/configs/llff.txt \
  --datadir /content/drive/MyDrive/data/nerf_llff_data/fortress \
  --ckpt /content/StyleTransfer/log/fortress/fortress.th \
  --render_only 1 \
  --render_test 1

Namespace(config='/content/StyleTransfer/configs/llff.txt', expname='trex', basedir='./log', add_timestamp=0, datadir='/content/drive/MyDrive/data/nerf_llff_data/fortress', wikiartdir='./data/WikiArt', progress_refresh_rate=10, with_depth=False, downsample_train=4.0, downsample_test=1.0, model_name='TensorVMSplit', batch_size=4096, n_iters=25000, dataset_name='llff', patch_size=256, chunk_size=4096, lr_init=0.02, lr_basis=0.0001, lr_finetune=1e-05, lr_decay_iters=-1, lr_decay_target_ratio=0.1, lr_upsample_reset=1, L1_weight_inital=0.0, L1_weight_rest=0, Ortho_weight=0.0, TV_weight_density=1.0, TV_weight_app=1.0, TV_weight_feature=0.0, style_weight=0, content_weight=0, image_tv_weight=0, featuremap_tv_weight=0, n_lamb_sigma=[16, 4, 4], n_lamb_sh=[48, 12, 12], data_dim_color=27, rm_weight_mask_thre=0.0001, alpha_mask_thre=0.0001, distance_scale=25, density_shift=-10, shadingMode='MLP_Fea', pos_pe=6, view_pe=0, fea_pe=0, featureC=128, ckpt='/content/StyleTransfer/log/fortress/fortress.th'

In [ ]:
!python /content/StyleTransfer/train_feature.py \
  --config /content/StyleTransfer/configs/llff_feature.txt \
  --datadir /content/drive/MyDrive/data/nerf_llff_data/fortress \
  --expname fortress_feature \
  --ckpt /content/StyleTransfer/log/fortress/fortress.th \
  --basedir /content/drive/MyDrive/StyleTransferResults


Namespace(config='/content/StyleTransfer/configs/llff_feature.txt', expname='fortress_feature', basedir='/content/drive/MyDrive/StyleTransferResults', add_timestamp=0, datadir='/content/drive/MyDrive/data/nerf_llff_data/fortress', wikiartdir='./data/WikiArt', progress_refresh_rate=10, with_depth=False, downsample_train=4.0, downsample_test=1.0, model_name='TensorVMSplit', batch_size=4096, n_iters=10000, dataset_name='llff', patch_size=256, chunk_size=4096, lr_init=0.02, lr_basis=0.0001, lr_finetune=1e-05, lr_decay_iters=-1, lr_decay_target_ratio=0.1, lr_upsample_reset=1, L1_weight_inital=0.0, L1_weight_rest=0, Ortho_weight=0.0, TV_weight_density=0.0, TV_weight_app=0.0, TV_weight_feature=80.0, style_weight=0, content_weight=0, image_tv_weight=0, featuremap_tv_weight=0, n_lamb_sigma=[16, 4, 4], n_lamb_sh=[48, 12, 12], data_dim_color=27, rm_weight_mask_thre=0.0001, alpha_mask_thre=0.0001, distance_scale=25, density_shift=-10, shadingMode='MLP_Fea', pos_pe=6, view_pe=6, fea_pe=6, featureC=

In [ ]:
!python /content/StyleTransfer/train_feature.py \
  --config /content/StyleTransfer/configs/llff_feature.txt \
  --datadir /content/drive/MyDrive/data/nerf_llff_data/fortress \
  --expname fortress_feature \
  --ckpt /content/drive/MyDrive/StyleTransferResults/fortress_feature/fortress_feature.th \
  --render_only 1 \
  --render_test 1 \
  --render_path 0 \
  --chunk_size 1024


Namespace(config='/content/StyleTransfer/configs/llff_feature.txt', expname='fortress_feature', basedir='./log_feature', add_timestamp=0, datadir='/content/drive/MyDrive/data/nerf_llff_data/fortress', wikiartdir='./data/WikiArt', progress_refresh_rate=10, with_depth=False, downsample_train=4.0, downsample_test=1.0, model_name='TensorVMSplit', batch_size=4096, n_iters=10000, dataset_name='llff', patch_size=256, chunk_size=1024, lr_init=0.02, lr_basis=0.0001, lr_finetune=1e-05, lr_decay_iters=-1, lr_decay_target_ratio=0.1, lr_upsample_reset=1, L1_weight_inital=0.0, L1_weight_rest=0, Ortho_weight=0.0, TV_weight_density=0.0, TV_weight_app=0.0, TV_weight_feature=80.0, style_weight=0, content_weight=0, image_tv_weight=0, featuremap_tv_weight=0, n_lamb_sigma=[16, 4, 4], n_lamb_sh=[48, 12, 12], data_dim_color=27, rm_weight_mask_thre=0.0001, alpha_mask_thre=0.0001, distance_scale=25, density_shift=-10, shadingMode='MLP_Fea', pos_pe=6, view_pe=6, fea_pe=6, featureC=128, ckpt='/content/drive/MyDr

In [ ]:
!python /content/StyleTransfer/train_style.py \
  --config /content/StyleTransfer/configs/llff_style.txt \
  --datadir /content/drive/MyDrive/data/nerf_llff_data/fortress \
  --wikiartdir /content/drive/MyDrive/data/WikiArt/images \
  --expname fortress_style \
  --ckpt /content/drive/MyDrive/StyleTransferResults/fortress_feature/fortress_feature.th

Namespace(config='/content/StyleTransfer/configs/llff_style.txt', expname='fortress_style', basedir='./log_style', add_timestamp=0, datadir='/content/drive/MyDrive/data/nerf_llff_data/fortress', wikiartdir='/content/drive/MyDrive/data/WikiArt/images', progress_refresh_rate=10, with_depth=False, downsample_train=4.0, downsample_test=1.0, model_name='TensorVMSplit', batch_size=4096, n_iters=25000, dataset_name='llff', patch_size=256, chunk_size=2048, lr_init=0.02, lr_basis=0.0001, lr_finetune=1e-05, lr_decay_iters=-1, lr_decay_target_ratio=0.1, lr_upsample_reset=1, L1_weight_inital=0.0, L1_weight_rest=0, Ortho_weight=0.0, TV_weight_density=0.0, TV_weight_app=0.0, TV_weight_feature=0.0, style_weight=20.0, content_weight=1.0, image_tv_weight=0.0, featuremap_tv_weight=0.0, n_lamb_sigma=[16, 4, 4], n_lamb_sh=[48, 12, 12], data_dim_color=27, rm_weight_mask_thre=0.001, alpha_mask_thre=0.0001, distance_scale=25, density_shift=-10, shadingMode='MLP_Fea', pos_pe=6, view_pe=6, fea_pe=6, featureC=1

In [ ]:
!cp -r /content/StyleTransfer/log_style/fortress_style /content/drive/MyDrive/StyleTransferResults/


In [ ]:
!python train_style.py \
  --config configs/llff_style.txt \
  --datadir /content/drive/MyDrive/data/nerf_llff_data/fortress \
  --expname fortress_style1 \
  --ckpt /content/drive/MyDrive/StyleTransferResults/fortress_style/fortress_style.th \
  --render_only 1 \
  --render_train 0 \
  --render_test 1 \
  --render_path 1 \
  --chunk_size 1024 \
  --rm_weight_mask_thre 0.0001 \
  --style_img /content/drive/MyDrive/data/WikiArt/images/landscape/00246e71f367b009f0aa7d0127b2bb08c.jpg


Namespace(config='configs/llff_style.txt', expname='fortress_style1', basedir='./log_style', add_timestamp=0, datadir='/content/drive/MyDrive/data/nerf_llff_data/fortress', wikiartdir='./data/WikiArt', progress_refresh_rate=10, with_depth=False, downsample_train=4.0, downsample_test=1.0, model_name='TensorVMSplit', batch_size=4096, n_iters=25000, dataset_name='llff', patch_size=256, chunk_size=1024, lr_init=0.02, lr_basis=0.0001, lr_finetune=1e-05, lr_decay_iters=-1, lr_decay_target_ratio=0.1, lr_upsample_reset=1, L1_weight_inital=0.0, L1_weight_rest=0, Ortho_weight=0.0, TV_weight_density=0.0, TV_weight_app=0.0, TV_weight_feature=0.0, style_weight=20.0, content_weight=1.0, image_tv_weight=0.0, featuremap_tv_weight=0.0, n_lamb_sigma=[16, 4, 4], n_lamb_sh=[48, 12, 12], data_dim_color=27, rm_weight_mask_thre=0.0001, alpha_mask_thre=0.0001, distance_scale=25, density_shift=-10, shadingMode='MLP_Fea', pos_pe=6, view_pe=6, fea_pe=6, featureC=128, ckpt='/content/drive/MyDrive/StyleTransferRes

In [ ]:
!python train_style.py \
  --config configs/llff_style.txt \
  --datadir /content/drive/MyDrive/data/nerf_llff_data/fortress \
  --expname fortress_style2 \
  --ckpt /content/drive/MyDrive/StyleTransferResults/fortress_style/fortress_style.th \
  --render_only 1 \
  --render_train 0 \
  --render_test 1 \
  --render_path 1 \
  --chunk_size 1024 \
  --rm_weight_mask_thre 0.0001 \
  --style_img /content/drive/MyDrive/data/WikiArt/images/landscape/00071f2b6fc24b1885ddb1b53b512081c.jpg


Namespace(config='configs/llff_style.txt', expname='fortress_style2', basedir='./log_style', add_timestamp=0, datadir='/content/drive/MyDrive/data/nerf_llff_data/fortress', wikiartdir='./data/WikiArt', progress_refresh_rate=10, with_depth=False, downsample_train=4.0, downsample_test=1.0, model_name='TensorVMSplit', batch_size=4096, n_iters=25000, dataset_name='llff', patch_size=256, chunk_size=1024, lr_init=0.02, lr_basis=0.0001, lr_finetune=1e-05, lr_decay_iters=-1, lr_decay_target_ratio=0.1, lr_upsample_reset=1, L1_weight_inital=0.0, L1_weight_rest=0, Ortho_weight=0.0, TV_weight_density=0.0, TV_weight_app=0.0, TV_weight_feature=0.0, style_weight=20.0, content_weight=1.0, image_tv_weight=0.0, featuremap_tv_weight=0.0, n_lamb_sigma=[16, 4, 4], n_lamb_sh=[48, 12, 12], data_dim_color=27, rm_weight_mask_thre=0.0001, alpha_mask_thre=0.0001, distance_scale=25, density_shift=-10, shadingMode='MLP_Fea', pos_pe=6, view_pe=6, fea_pe=6, featureC=128, ckpt='/content/drive/MyDrive/StyleTransferRes

In [ ]:
!zip -r fortress_base.zip /content/StyleTransfer/log/fortress
from google.colab import files
files.download("fortress_base.zip")


  adding: content/StyleTransfer/log/fortress/ (stored 0%)
  adding: content/StyleTransfer/log/fortress/trex/ (stored 0%)
  adding: content/StyleTransfer/log/fortress/trex/imgs_path_all/ (stored 0%)
  adding: content/StyleTransfer/log/fortress/trex/imgs_path_all/103.png (deflated 0%)
  adding: content/StyleTransfer/log/fortress/trex/imgs_path_all/024.png (deflated 0%)
  adding: content/StyleTransfer/log/fortress/trex/imgs_path_all/075.png (deflated 0%)
  adding: content/StyleTransfer/log/fortress/trex/imgs_path_all/060.png (deflated 0%)
  adding: content/StyleTransfer/log/fortress/trex/imgs_path_all/073.png (deflated 0%)
  adding: content/StyleTransfer/log/fortress/trex/imgs_path_all/019.png (deflated 0%)
  adding: content/StyleTransfer/log/fortress/trex/imgs_path_all/081.png (deflated 0%)
  adding: content/StyleTransfer/log/fortress/trex/imgs_path_all/014.png (deflated 0%)
  adding: content/StyleTransfer/log/fortress/trex/imgs_path_all/071.png (deflated 0%)
  adding: content/StyleTrans

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!python train_feature.py \
  --config configs/llff_feature.txt \
  --datadir /content/drive/MyDrive/data/nerf_llff_data/fortress \
  --expname trex_feature_lowmem12 \
  --ckpt /content/drive/MyDrive/trex_base/fortress_base.th


Namespace(config='configs/llff_feature.txt', expname='trex_feature_lowmem12', basedir='./log_feature', add_timestamp=0, datadir='/content/drive/MyDrive/data/nerf_llff_data/trex', wikiartdir='./data/WikiArt', progress_refresh_rate=10, with_depth=False, downsample_train=4.0, downsample_test=1.0, model_name='TensorVMSplit', batch_size=4096, n_iters=10000, dataset_name='llff', patch_size=256, chunk_size=4096, lr_init=0.02, lr_basis=0.0001, lr_finetune=1e-05, lr_decay_iters=-1, lr_decay_target_ratio=0.1, lr_upsample_reset=1, L1_weight_inital=0.0, L1_weight_rest=0, Ortho_weight=0.0, TV_weight_density=0.0, TV_weight_app=0.0, TV_weight_feature=80.0, style_weight=0, content_weight=0, image_tv_weight=0, featuremap_tv_weight=0, n_lamb_sigma=[16, 4, 4], n_lamb_sh=[48, 12, 12], data_dim_color=27, rm_weight_mask_thre=0.0001, alpha_mask_thre=0.0001, distance_scale=25, density_shift=-10, shadingMode='MLP_Fea', pos_pe=6, view_pe=6, fea_pe=6, featureC=128, ckpt='/content/drive/MyDrive/trex_base/trex_bas

In [ ]:
!zip -r log_feature12.zip /content/StyleTransfer/log_feature/trex_feature_lowmem12
from google.colab import files
files.download("log_feature12.zip")

updating: content/StyleTransfer/log_feature/trex_feature_lowmem12/ (stored 0%)
updating: content/StyleTransfer/log_feature/trex_feature_lowmem12/trex_feature_lowmem12.th (deflated 9%)
updating: content/StyleTransfer/log_feature/trex_feature_lowmem12/events.out.tfevents.1743696393.d432122587d1 (deflated 13%)
updating: content/StyleTransfer/log_feature/trex_feature_lowmem12/events.out.tfevents.1743723046.d432122587d1 (deflated 13%)
updating: content/StyleTransfer/log_feature/trex_feature_lowmem12/events.out.tfevents.1743696233.d432122587d1 (deflated 1%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!python train_style.py \
  --config=configs/llff_style.txt \
  --datadir /content/drive/MyDrive/data/nerf_llff_data/trex \
  --wikiartdir /content/drive/MyDrive/data/WikiArt/images \
  --expname trex_style_landscape \
  --ckpt /content/StyleTransfer/log_feature/trex_feature_lowmem12/trex_feature_lowmem12.th

Namespace(config='configs/llff_style.txt', expname='trex_style_landscape', basedir='./log_style', add_timestamp=0, datadir='/content/drive/MyDrive/data/nerf_llff_data/trex', wikiartdir='/content/drive/MyDrive/data/WikiArt/images', progress_refresh_rate=10, with_depth=False, downsample_train=4.0, downsample_test=1.0, model_name='TensorVMSplit', batch_size=4096, n_iters=10000, dataset_name='llff', patch_size=256, chunk_size=2048, lr_init=0.02, lr_basis=0.0001, lr_finetune=1e-05, lr_decay_iters=-1, lr_decay_target_ratio=0.1, lr_upsample_reset=1, L1_weight_inital=0.0, L1_weight_rest=0, Ortho_weight=0.0, TV_weight_density=0.0, TV_weight_app=0.0, TV_weight_feature=0.0, style_weight=20.0, content_weight=1.0, image_tv_weight=0.0, featuremap_tv_weight=0.0, n_lamb_sigma=[16, 4, 4], n_lamb_sh=[48, 12, 12], data_dim_color=27, rm_weight_mask_thre=0.001, alpha_mask_thre=0.0001, distance_scale=25, density_shift=-10, shadingMode='MLP_Fea', pos_pe=6, view_pe=6, fea_pe=6, featureC=128, ckpt='/content/St

In [ ]:
!python train_style.py \
  --config=configs/llff_style.txt \
  --datadir /content/drive/MyDrive/data/nerf_llff_data/trex \
  --wikiartdir /content/drive/MyDrive/data/WikiArt/images \
  --expname trex_style_landscape \
  --ckpt /content/drive/MyDrive/log_feature1/trex_feature_lowmem12/trex_feature_lowmem12.th

Namespace(config='configs/llff_style.txt', expname='trex_style_landscape', basedir='./log_style', add_timestamp=0, datadir='/content/drive/MyDrive/data/nerf_llff_data/trex', wikiartdir='/content/drive/MyDrive/data/WikiArt/images', progress_refresh_rate=10, with_depth=False, downsample_train=4.0, downsample_test=1.0, model_name='TensorVMSplit', batch_size=4096, n_iters=10000, dataset_name='llff', patch_size=256, chunk_size=2048, lr_init=0.02, lr_basis=0.0001, lr_finetune=1e-05, lr_decay_iters=-1, lr_decay_target_ratio=0.1, lr_upsample_reset=1, L1_weight_inital=0.0, L1_weight_rest=0, Ortho_weight=0.0, TV_weight_density=0.0, TV_weight_app=0.0, TV_weight_feature=0.0, style_weight=20.0, content_weight=1.0, image_tv_weight=0.0, featuremap_tv_weight=0.0, n_lamb_sigma=[16, 4, 4], n_lamb_sh=[48, 12, 12], data_dim_color=27, rm_weight_mask_thre=0.001, alpha_mask_thre=0.0001, distance_scale=25, density_shift=-10, shadingMode='MLP_Fea', pos_pe=6, view_pe=6, fea_pe=6, featureC=128, ckpt='/content/dr

In [ ]:
!python train_style.py \
  --config=configs/llff_style.txt \
  --datadir /content/drive/MyDrive/data/nerf_llff_data/trex \
  --wikiartdir /content/drive/MyDrive/data/WikiArt/images \
  --expname trex_style_landscape2 \
  --ckpt /content/drive/MyDrive/log_feature1/trex_feature_lowmem12/trex_feature_lowmem12.th

Namespace(config='configs/llff_style.txt', expname='trex_style_landscape2', basedir='./log_style', add_timestamp=0, datadir='/content/drive/MyDrive/data/nerf_llff_data/trex', wikiartdir='/content/drive/MyDrive/data/WikiArt/images', progress_refresh_rate=10, with_depth=False, downsample_train=4.0, downsample_test=1.0, model_name='TensorVMSplit', batch_size=4096, n_iters=10000, dataset_name='llff', patch_size=256, chunk_size=2048, lr_init=0.02, lr_basis=0.0001, lr_finetune=1e-05, lr_decay_iters=-1, lr_decay_target_ratio=0.1, lr_upsample_reset=1, L1_weight_inital=0.0, L1_weight_rest=0, Ortho_weight=0.0, TV_weight_density=0.0, TV_weight_app=0.0, TV_weight_feature=0.0, style_weight=20.0, content_weight=1.0, image_tv_weight=0.0, featuremap_tv_weight=0.0, n_lamb_sigma=[16, 4, 4], n_lamb_sh=[48, 12, 12], data_dim_color=27, rm_weight_mask_thre=0.001, alpha_mask_thre=0.0001, distance_scale=25, density_shift=-10, shadingMode='MLP_Fea', pos_pe=6, view_pe=6, fea_pe=6, featureC=128, ckpt='/content/d

In [ ]:
!python train_style.py \
  --config=configs/llff_style.txt \
  --datadir /content/drive/MyDrive/data/nerf_llff_data/trex \
  --wikiartdir /content/drive/MyDrive/data/WikiArt/images \
  --expname trex_style_landscape2 \
  --ckpt /content/drive/MyDrive/log_feature1/trex_feature_lowmem12/trex_feature_lowmem12.th

In [ ]:
!zip -r trex_style_landscape2.zip /content/StyleTransfer/log_style/trex_style_landscape2

from google.colab import files
files.download("trex_style_landscape2.zip")

updating: content/StyleTransfer/log_style/trex_style_landscape2/ (stored 0%)
updating: content/StyleTransfer/log_style/trex_style_landscape2/events.out.tfevents.1743778974.f65652252e16 (deflated 5%)
updating: content/StyleTransfer/log_style/trex_style_landscape2/events.out.tfevents.1743779312.f65652252e16 (deflated 1%)
updating: content/StyleTransfer/log_style/trex_style_landscape2/events.out.tfevents.1743778688.f65652252e16 (deflated 5%)
  adding: content/StyleTransfer/log_style/trex_style_landscape2/trex_style_landscape2.th (deflated 9%)
  adding: content/StyleTransfer/log_style/trex_style_landscape2/events.out.tfevents.1743807552.f65652252e16 (deflated 1%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!zip -r trex_style_landscape.zip /content/StyleTransfer/log_style/trex_style_landscape

from google.colab import files
files.download("trex_style_landscape.zip")

  adding: content/StyleTransfer/log_style/trex_style_landscape/ (stored 0%)
  adding: content/StyleTransfer/log_style/trex_style_landscape/trex_style_landscape.th (deflated 9%)
  adding: content/StyleTransfer/log_style/trex_style_landscape/events.out.tfevents.1743753470.df11ce55fb23 (deflated 1%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!python train.py \
  --config configs/llff.txt \
  --datadir /content/drive/MyDrive/data/nerf_llff_data/trex \
  --ckpt /content/drive/MyDrive/trex_base/trex_base.th \
  --render_only 1 \
  --render_test 1


Namespace(config='configs/llff.txt', expname='trex', basedir='./log', add_timestamp=0, datadir='/content/drive/MyDrive/data/nerf_llff_data/trex', wikiartdir='./data/WikiArt', progress_refresh_rate=10, with_depth=False, downsample_train=4.0, downsample_test=1.0, model_name='TensorVMSplit', batch_size=4096, n_iters=25000, dataset_name='llff', patch_size=256, chunk_size=4096, lr_init=0.02, lr_basis=0.0001, lr_finetune=1e-05, lr_decay_iters=-1, lr_decay_target_ratio=0.1, lr_upsample_reset=1, L1_weight_inital=0.0, L1_weight_rest=0, Ortho_weight=0.0, TV_weight_density=1.0, TV_weight_app=1.0, TV_weight_feature=0.0, style_weight=0, content_weight=0, image_tv_weight=0, featuremap_tv_weight=0, n_lamb_sigma=[16, 4, 4], n_lamb_sh=[48, 12, 12], data_dim_color=27, rm_weight_mask_thre=0.0001, alpha_mask_thre=0.0001, distance_scale=25, density_shift=-10, shadingMode='MLP_Fea', pos_pe=6, view_pe=0, fea_pe=0, featureC=128, ckpt='/content/drive/MyDrive/trex_base/trex_base.th', render_only=1, render_test=

In [ ]:
!python train_feature.py \
  --config configs/llff_feature.txt \
  --datadir /content/drive/MyDrive/data/nerf_llff_data/trex \
  --expname trex_feature_lowmem22 \
  --ckpt /content/drive/MyDrive/log_feature1/trex_feature_lowmem12/trex_feature_lowmem12.th \
  --render_only 1 \
  --render_test 1 \
  --render_path 0 \
  --chunk_size 1024


Namespace(config='configs/llff_feature.txt', expname='trex_feature_lowmem22', basedir='./log_feature', add_timestamp=0, datadir='/content/drive/MyDrive/data/nerf_llff_data/trex', wikiartdir='./data/WikiArt', progress_refresh_rate=10, with_depth=False, downsample_train=4.0, downsample_test=1.0, model_name='TensorVMSplit', batch_size=4096, n_iters=25000, dataset_name='llff', patch_size=256, chunk_size=1024, lr_init=0.02, lr_basis=0.0001, lr_finetune=1e-05, lr_decay_iters=-1, lr_decay_target_ratio=0.1, lr_upsample_reset=1, L1_weight_inital=0.0, L1_weight_rest=0, Ortho_weight=0.0, TV_weight_density=0.0, TV_weight_app=0.0, TV_weight_feature=80.0, style_weight=0, content_weight=0, image_tv_weight=0, featuremap_tv_weight=0, n_lamb_sigma=[16, 4, 4], n_lamb_sh=[48, 12, 12], data_dim_color=27, rm_weight_mask_thre=0.0001, alpha_mask_thre=0.0001, distance_scale=25, density_shift=-10, shadingMode='MLP_Fea', pos_pe=6, view_pe=6, fea_pe=6, featureC=128, ckpt='/content/drive/MyDrive/log_feature1/trex_

In [ ]:
!python train_style.py \
  --config configs/llff_style.txt \
  --datadir /content/drive/MyDrive/data/nerf_llff_data/trex \
  --expname trex_style_landscape \
  --ckpt /content/StyleTransfer/log_style/trex_style_landscape/trex_style_landscape.th \
  --render_only 1 \
  --render_train 0 \
  --render_test 1 \
  --render_path 1 \
  --chunk_size 1024 \
  --rm_weight_mask_thre 0.0001 \
  --style_img /content/drive/MyDrive/data/WikiArt/images/landscape/00246e71f367b009f0aa7d0127b2bb08c.jpg  # 修改成你的图片路径


Namespace(config='configs/llff_style.txt', expname='trex_style_landscape', basedir='./log_style', add_timestamp=0, datadir='/content/drive/MyDrive/data/nerf_llff_data/trex', wikiartdir='./data/WikiArt', progress_refresh_rate=10, with_depth=False, downsample_train=4.0, downsample_test=1.0, model_name='TensorVMSplit', batch_size=4096, n_iters=10000, dataset_name='llff', patch_size=256, chunk_size=1024, lr_init=0.02, lr_basis=0.0001, lr_finetune=1e-05, lr_decay_iters=-1, lr_decay_target_ratio=0.1, lr_upsample_reset=1, L1_weight_inital=0.0, L1_weight_rest=0, Ortho_weight=0.0, TV_weight_density=0.0, TV_weight_app=0.0, TV_weight_feature=0.0, style_weight=20.0, content_weight=1.0, image_tv_weight=0.0, featuremap_tv_weight=0.0, n_lamb_sigma=[16, 4, 4], n_lamb_sh=[48, 12, 12], data_dim_color=27, rm_weight_mask_thre=0.0001, alpha_mask_thre=0.0001, distance_scale=25, density_shift=-10, shadingMode='MLP_Fea', pos_pe=6, view_pe=6, fea_pe=6, featureC=128, ckpt='/content/StyleTransfer/log_style/trex_

In [ ]:
!python train_style.py \
  --config configs/llff_style.txt \
  --datadir /content/drive/MyDrive/data/nerf_llff_data/trex \
  --expname trex_style_landscape \
  --ckpt /content/drive/MyDrive/log_style/trex_style_landscape/trex_style_landscape.th \
  --render_only 1 \
  --render_train 0 \
  --render_test 1 \
  --render_path 1 \
  --chunk_size 512 \
  --patch_size 128 \
  --nSamples 256 \
  --N_vis 3 \
  --rm_weight_mask_thre 0.0001 \
  --style_img /content/drive/MyDrive/data/WikiArt/images/landscape/00246e71f367b009f0aa7d0127b2bb08c.jpg  # 修改成你的图片路径


Namespace(config='configs/llff_style.txt', expname='trex_style_landscape', basedir='./log_style', add_timestamp=0, datadir='/content/drive/MyDrive/data/nerf_llff_data/trex', wikiartdir='./data/WikiArt', progress_refresh_rate=10, with_depth=False, downsample_train=4.0, downsample_test=1.0, model_name='TensorVMSplit', batch_size=4096, n_iters=10000, dataset_name='llff', patch_size=128, chunk_size=512, lr_init=0.02, lr_basis=0.0001, lr_finetune=1e-05, lr_decay_iters=-1, lr_decay_target_ratio=0.1, lr_upsample_reset=1, L1_weight_inital=0.0, L1_weight_rest=0, Ortho_weight=0.0, TV_weight_density=0.0, TV_weight_app=0.0, TV_weight_feature=0.0, style_weight=20.0, content_weight=1.0, image_tv_weight=0.0, featuremap_tv_weight=0.0, n_lamb_sigma=[16, 4, 4], n_lamb_sh=[48, 12, 12], data_dim_color=27, rm_weight_mask_thre=0.0001, alpha_mask_thre=0.0001, distance_scale=25, density_shift=-10, shadingMode='MLP_Fea', pos_pe=6, view_pe=6, fea_pe=6, featureC=128, ckpt='/content/drive/MyDrive/log_style/trex_s

In [ ]:
!python train_style.py \
  --config configs/llff_style.txt \
  --datadir /content/drive/MyDrive/data/nerf_llff_data/trex \
  --expname trex_style_landscape \
  --ckpt /content/drive/MyDrive/log_style/trex_style_landscape/trex_style_landscape.th \
  --render_only 1 \
  --render_train 0 \
  --render_test 1 \
  --render_path 1 \
  --chunk_size 1024 \
  --rm_weight_mask_thre 0.0001 \
  --style_img /content/drive/MyDrive/data/WikiArt/images/landscape/00246e71f367b009f0aa7d0127b2bb08c.jpg  # 修改成你的图片路径


Namespace(config='configs/llff_style.txt', expname='trex_style_landscape', basedir='./log_style', add_timestamp=0, datadir='/content/drive/MyDrive/data/nerf_llff_data/trex', wikiartdir='./data/WikiArt', progress_refresh_rate=10, with_depth=False, downsample_train=4.0, downsample_test=1.0, model_name='TensorVMSplit', batch_size=4096, n_iters=25000, dataset_name='llff', patch_size=256, chunk_size=1024, lr_init=0.02, lr_basis=0.0001, lr_finetune=1e-05, lr_decay_iters=-1, lr_decay_target_ratio=0.1, lr_upsample_reset=1, L1_weight_inital=0.0, L1_weight_rest=0, Ortho_weight=0.0, TV_weight_density=0.0, TV_weight_app=0.0, TV_weight_feature=0.0, style_weight=20.0, content_weight=1.0, image_tv_weight=0.0, featuremap_tv_weight=0.0, n_lamb_sigma=[16, 4, 4], n_lamb_sh=[48, 12, 12], data_dim_color=27, rm_weight_mask_thre=0.0001, alpha_mask_thre=0.0001, distance_scale=25, density_shift=-10, shadingMode='MLP_Fea', pos_pe=6, view_pe=6, fea_pe=6, featureC=128, ckpt='/content/drive/MyDrive/log_style/trex_

In [ ]:
!python train_style.py \
  --config configs/llff_style.txt \
  --datadir /content/drive/MyDrive/data/nerf_llff_data/trex \
  --expname trex_style_landscape \
  --ckpt /content/StyleTransfer/log_style/trex_style_landscape2/trex_style_landscape2.th \
  --render_only 1 \
  --render_train 0 \
  --render_test 1 \
  --render_path 1 \
  --chunk_size 1024 \
  --rm_weight_mask_thre 0.0001 \
  --style_img /content/drive/MyDrive/data/WikiArt/images/landscape/00246e71f367b009f0aa7d0127b2bb08c.jpg  # 修改成你的图片路径


Namespace(config='configs/llff_style.txt', expname='trex_style_landscape', basedir='./log_style', add_timestamp=0, datadir='/content/drive/MyDrive/data/nerf_llff_data/trex', wikiartdir='./data/WikiArt', progress_refresh_rate=10, with_depth=False, downsample_train=4.0, downsample_test=1.0, model_name='TensorVMSplit', batch_size=4096, n_iters=10000, dataset_name='llff', patch_size=256, chunk_size=1024, lr_init=0.02, lr_basis=0.0001, lr_finetune=1e-05, lr_decay_iters=-1, lr_decay_target_ratio=0.1, lr_upsample_reset=1, L1_weight_inital=0.0, L1_weight_rest=0, Ortho_weight=0.0, TV_weight_density=0.0, TV_weight_app=0.0, TV_weight_feature=0.0, style_weight=20.0, content_weight=1.0, image_tv_weight=0.0, featuremap_tv_weight=0.0, n_lamb_sigma=[16, 4, 4], n_lamb_sh=[48, 12, 12], data_dim_color=27, rm_weight_mask_thre=0.0001, alpha_mask_thre=0.0001, distance_scale=25, density_shift=-10, shadingMode='MLP_Fea', pos_pe=6, view_pe=6, fea_pe=6, featureC=128, ckpt='/content/StyleTransfer/log_style/trex_

In [ ]:
!python train_style.py \
  --config configs/llff_style.txt \
  --datadir /content/drive/MyDrive/data/nerf_llff_data/trex \
  --expname trex_style_landscape \
  --ckpt /content/drive/MyDrive/log_style/trex_style_landscape2/trex_style_landscape2.th \
  --render_only 1 \
  --render_train 0 \
  --render_test 1 \
  --render_path 1 \
  --chunk_size 1024 \
  --rm_weight_mask_thre 0.0001 \
  --style_img /content/drive/MyDrive/data/WikiArt/images/landscape/10d57f303904c2352a7410ce7c2dfb9fc.jpg


Namespace(config='configs/llff_style.txt', expname='trex_style_landscape', basedir='./log_style', add_timestamp=0, datadir='/content/drive/MyDrive/data/nerf_llff_data/trex', wikiartdir='./data/WikiArt', progress_refresh_rate=10, with_depth=False, downsample_train=4.0, downsample_test=1.0, model_name='TensorVMSplit', batch_size=4096, n_iters=10000, dataset_name='llff', patch_size=256, chunk_size=1024, lr_init=0.02, lr_basis=0.0001, lr_finetune=1e-05, lr_decay_iters=-1, lr_decay_target_ratio=0.1, lr_upsample_reset=1, L1_weight_inital=0.0, L1_weight_rest=0, Ortho_weight=0.0, TV_weight_density=0.0, TV_weight_app=0.0, TV_weight_feature=0.0, style_weight=20.0, content_weight=1.0, image_tv_weight=0.0, featuremap_tv_weight=0.0, n_lamb_sigma=[16, 4, 4], n_lamb_sh=[48, 12, 12], data_dim_color=27, rm_weight_mask_thre=0.0001, alpha_mask_thre=0.0001, distance_scale=25, density_shift=-10, shadingMode='MLP_Fea', pos_pe=6, view_pe=6, fea_pe=6, featureC=128, ckpt='/content/drive/MyDrive/log_style/trex_

In [ ]:
!python train_style.py \
  --config configs/llff_style.txt \
  --datadir /content/drive/MyDrive/data/nerf_llff_data/trex \
  --expname trex_style_landscape3 \
  --ckpt /content/drive/MyDrive/log_style/trex_style_landscape2/trex_style_landscape2.th \
  --render_only 1 \
  --render_train 0 \
  --render_test 1 \
  --render_path 1 \
  --chunk_size 1024 \
  --rm_weight_mask_thre 0.0001 \
  --style_img /content/drive/MyDrive/data/WikiArt/images/marina/0020ab920f097f6f604aafcc557c42ebc.jpg


Namespace(config='configs/llff_style.txt', expname='trex_style_landscape3', basedir='./log_style', add_timestamp=0, datadir='/content/drive/MyDrive/data/nerf_llff_data/trex', wikiartdir='./data/WikiArt', progress_refresh_rate=10, with_depth=False, downsample_train=4.0, downsample_test=1.0, model_name='TensorVMSplit', batch_size=4096, n_iters=10000, dataset_name='llff', patch_size=256, chunk_size=1024, lr_init=0.02, lr_basis=0.0001, lr_finetune=1e-05, lr_decay_iters=-1, lr_decay_target_ratio=0.1, lr_upsample_reset=1, L1_weight_inital=0.0, L1_weight_rest=0, Ortho_weight=0.0, TV_weight_density=0.0, TV_weight_app=0.0, TV_weight_feature=0.0, style_weight=20.0, content_weight=1.0, image_tv_weight=0.0, featuremap_tv_weight=0.0, n_lamb_sigma=[16, 4, 4], n_lamb_sh=[48, 12, 12], data_dim_color=27, rm_weight_mask_thre=0.0001, alpha_mask_thre=0.0001, distance_scale=25, density_shift=-10, shadingMode='MLP_Fea', pos_pe=6, view_pe=6, fea_pe=6, featureC=128, ckpt='/content/drive/MyDrive/log_style/trex